In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools

include_columns = [0, 1, 2, 3, 7, 10, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23] 
movies = pd.read_csv('Datafiles\\movies_metadata.csv', sep = ',', low_memory=False, \
        usecols = include_columns)
movies.info()

1. Data filtering

In [ ]:
f_movies = movies[movies['adult'] == 'False']
f_movies = f_movies[f_movies['status'] == 'Released']
f_movies = f_movies[f_movies['video'] == False]

In [ ]:
f_movies = f_movies.drop(['adult', 'status', 'video'], axis=1)

2. Data wrangling
BELONGS TO COLLECTION

In [ ]:
f_movies.loc[f_movies['belongs_to_collection'].notna(), 'belongs_to_collection'] = 1
f_movies.loc[f_movies['belongs_to_collection'].isna(), 'belongs_to_collection'] = 0


In [ ]:
f_movies['belongs_to_collection'] = f_movies['belongs_to_collection'].astype(int)
f_movies['belongs_to_collection'].value_counts()
#f_movies['belongs_to_collection'].dtype


In [ ]:
f_movies['budget'] = f_movies['budget'].astype(int)
f_movies['budget'].value_counts()
#plt.hist(f_movies['budget'])
budget_nonzero = f_movies[f_movies['budget'] != 0]['budget']
plt.hist(budget_nonzero)



In [ ]:
plt.boxplot(budget_nonzero)
#budget_nonzero

In [ ]:
f_movies['original_language'] = f_movies['original_language'].astype(str)
lang = f_movies['original_language'].value_counts()
lang_other = lang[lang <= 100]
f_movies.loc[f_movies['original_language'].isin(lang_other.index.to_list()), 'original_language'] = 'other'
f_movies['original_language'].value_counts()

In [ ]:
f_movies['popularity'] = f_movies['popularity'].astype(float)
#plt.hist(f_movies['popularity'])
#f_movies['popularity'].describe()
#f_movies['popularity'].sort_values(ascending=False).head(20)
f_movies[f_movies['popularity'] > 200].sort_values(by='popularity', ascending = False)
pop = f_movies['popularity']


def normalizer(vect, a, b):
    min_vect = min(vect)
    max_vect = max(vect)
    result = ((b - a)*(vect - min_vect)/(max_vect-min_vect)) + a
    return result

pop_stand = normalizer(pop, 0, 100)
pop_stand.describe()
#plt.hist(pop_stand)

RELEASE DATE

In [ ]:
f_movies['release_date'] = f_movies['release_date'].astype(str)
f_movies.loc[f_movies['release_date'] == 'nan', 'release_date'] = '0000'
f_movies['release_date'] = f_movies['release_date'].str[:4]

f_movies['release_date'] = pd.to_numeric(f_movies['release_date'])
plt.hist(f_movies[f_movies['release_date'] != 0]['release_date'])


REVENUE

In [ ]:
f_movies['revenue'].value_counts()
plt.hist(f_movies[f_movies['revenue'] != 0]['revenue'])

In [ ]:
plt.boxplot(f_movies[f_movies['revenue'] != 0]['revenue'])

RUNTIME

In [ ]:
f_movies['runtime'].value_counts()
f_movies['runtime'].describe()

#plt.hist(f_movies[f_movies['runtime'] != 0]['runtime'])
f_movies['runtime'].sort_values(ascending=False).head(200)


In [ ]:
f_movies['vote_count'].value_counts()
f_movies['vote_average'].value_counts()


In [ ]:
import json
"""
converts from string to dictionary and extracts values from particulary key
"""
final_list = []
idx = 0
for row in f_movies['genres']:
    if row == '[]':
        final_list.append(['Unknown'])
    else:
        final_values = []
        row = row.strip('[]').replace("'", "\"")
        for i in range(len(row)):
            if row[i] == ',':
                if row[i-1] != '}':
                    pass
                else:
                    row = row[:i] + '@' + row[i + 1:]
        row = row.replace(" ", "").split('@')
        # print(idx)
        # print(row)
        idx += 1
        for item in row: #for each dictionary in a row
            tmp = json.loads(item) #load that dict
            final_values.append(tmp['name']) #extract value
        final_list.append(final_values)
f_movies['genres'] = final_list

In [ ]:
import itertools
np.unique([*itertools.chain.from_iterable(f_movies.genres)])

In [ ]:
s = f_movies['genres'].explode()
f_movies = f_movies.drop('genres', axis = 1).join(pd.crosstab(s.index, s))

In [ ]:
f_movies['release_date'].value_counts()
#f_movies[f_movies['release_date'] == 0]

In [ ]:
f_movies = f_movies[f_movies['vote_count'] != 0]
f_movies = f_movies[f_movies['release_date'] != 0]